In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from fastprogress.fastprogress import master_bar, progress_bar

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SequenceModel(nn.Module):
    def __init__(self):
        super(SequenceModel, self).__init__()
        self.num_positions = 9
        self.num_embeddings = 1024
        self.embedding_dim = 6
        
        # Embedding layers for each position
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=self.num_embeddings, embedding_dim=self.embedding_dim) 
            for _ in range(self.num_positions)
        ])
        
        # 1D Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=54, out_channels=54, kernel_size=5, stride=4)
        self.conv2 = nn.Conv1d(in_channels=54, out_channels=54, kernel_size=5, stride=4)
        self.conv3 = nn.Conv1d(in_channels=54, out_channels=54, kernel_size=5, stride=4)
        self.conv4 = nn.Conv1d(in_channels=54, out_channels=54, kernel_size=5, stride=4)
        self.conv5 = nn.Conv1d(in_channels=54, out_channels=27, kernel_size=3, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(243, 81)
        self.fc2 = nn.Linear(81, 24)
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        # Apply embeddings in parallel
        embeddings = [self.embeddings[i](x[:, :, i]) for i in range(self.num_positions)]
        embeddings = torch.stack(embeddings, dim=3)
        
        # Reshape: combining the last two dimensions
        reshaped = embeddings.view(batch_size, 54, -1)
        
        # Apply convolutional layers
        x = F.relu(self.conv1(reshaped))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        
        # Reshape to a flat vector for the fully connected layers
        x = x.view(batch_size, -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

# Create the model instance
model = SequenceModel()

# Example input: a batch of 2 sequences, each of length 1152, with 9-dimensional, 10-bit codes
example_input = torch.randint(0, 1024, (2, 1152, 9), dtype=torch.long)

# Forward pass
output = model(example_input)

print(output.shape)  # Expected shape: [2, 24] for batch size of 2 and 24 output classes
